In [1]:
import SimpleITK as sitk
import numpy as np
import os

# Folder name where registration transforms are saved
folder = "TestDenoising5"

path = "./oai_zib_mri_train/train"
labelPath = "oai_zib_labelmaps/labelmaps/train"

P = [3, 3, 3]
N = [5, 5, 5]

In [2]:
# Read boundaries
file1 = open(f"./{folder}/boundaries.txt", "r")
temp = file1.read().split("\n")[:-1]
minx = int(temp[0])
maxx = int(temp[1])
miny = int(temp[2])
maxy = int(temp[3])
minz = int(temp[4])
maxz = int(temp[5])

In [3]:
file = open(f"./{folder}/names.txt", "r") 
name = file.read().split("\n")[0]

pathName = os.path.join(path, name)
images = []
images.append(sitk.GetArrayFromImage(sitk.ReadImage(pathName)))

shape = [[minx - (P[0]//2 + N[0]//2), maxx + (P[0]//2 + N[0]//2)],
         [miny - (P[1]//2 + N[1]//2), maxy + (P[1]//2 + N[1]//2)],
         [minz - (P[2]//2 + N[2]//2), maxz + (P[2]//2 + N[2]//2)],
        ]

copyShape = [[],[],[]]

# x range
if shape[0][0] < 0: 
    copyShape[0].append(0)
else:
    copyShape[0].append(shape[0][0])
    
if shape[0][1] > images[0].shape[0] - 1: 
    copyShape[0].append(images[0].shape[0] - 1)
else:
    copyShape[0].append(shape[0][1])

# y range
if shape[1][0] < 0: 
    copyShape[1].append(0)
else:
    copyShape[1].append(shape[1][0])
    
if shape[1][1] > images[0].shape[1] - 1: 
    copyShape[1].append(images[0].shape[1] - 1)
else:
    copyShape[1].append(shape[1][1])
    
#z range
if shape[2][0] < 0: 
    copyShape[2].append(0)
else:
    copyShape[2].append(shape[2][0])
    
if shape[2][1] > images[0].shape[2] - 1: 
    copyShape[2].append(images[0].shape[2] - 1)
else:
    copyShape[2].append(shape[2][1])
    
offset = []
offset.append(copyShape[0][0] - shape[0][0])
offset.append(copyShape[1][0] - shape[1][0])
offset.append(copyShape[2][0] - shape[2][0])

length = []
length.append(copyShape[0][1] - copyShape[0][0] + 1)
length.append(copyShape[1][1] - copyShape[1][0] + 1)
length.append(copyShape[2][1] - copyShape[2][0] + 1)

del images[0]
print(images)

[]


In [4]:
"""Read images and labels and make them numpy images with the proper size."""
from utils import *

images = []
labels = []

file = open(f"./{folder}/names.txt", "r") 
imageNames = file.read().split("\n")[:-1]
# del imageNames[1]

for i, name in enumerate(imageNames[:3]):
    print(f'Reading {name}. {i+1}/{len(imageNames)}')
    
    pathName = os.path.join(path, name)
    
    if i >= 1:
        t = sitk.ReadTransform(f"./{folder}/{name}.tfm")
        
        image = sitk.GetArrayFromImage(resampleImage(sitk.HistogramMatching(sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                                                                               timeStep = 0.04, 
                                                                                               numberOfIterations = 10), 
                                                                            images[0]), images[0], t))

        newImage = np.zeros((shape[0][1] - shape[0][0] + 1, shape[1][1] - shape[1][0] + 1, shape[2][1] - shape[2][0] + 1), dtype="uint16", order="F")
        newImage[offset[0]:offset[0]+length[0], 
                 offset[1]:offset[1]+length[1], 
                 offset[2]:offset[2]+length[2]] = image[copyShape[0][0]:copyShape[0][1]+1,
                                                        copyShape[1][0]:copyShape[1][1]+1,
                                                        copyShape[2][0]:copyShape[2][1]+1,]
        images.append(newImage)
        
        label = sitk.GetArrayFromImage(resampleSegmentation(sitk.ReadImage(os.path.join(labelPath, name)), images[0], t))
        
        newLabels = np.zeros((shape[0][1] - shape[0][0] + 1, shape[1][1] - shape[1][0] + 1, shape[2][1] - shape[2][0] + 1,), dtype="uint8", order="F")
        newLabels[offset[0]:offset[0]+length[0], 
                  offset[1]:offset[1]+length[1], 
                  offset[2]:offset[2]+length[2]] = label[copyShape[0][0]:copyShape[0][1]+1,
                                                         copyShape[1][0]:copyShape[1][1]+1,
                                                         copyShape[2][0]:copyShape[2][1]+1,]
        labels.append(newLabels)
    else:
        images.append(sitk.CurvatureFlow(sitk.ReadImage(pathName), timeStep = 0.04, numberOfIterations = 10))

        labels.append(sitk.ReadImage(os.path.join(labelPath, name)))
        
images[0] = sitk.GetArrayFromImage(images[0])
newImage = np.zeros((shape[0][1] - shape[0][0] + 1, shape[1][1] - shape[1][0] + 1, shape[2][1] - shape[2][0] + 1), dtype="uint16", order="F")
newImage[offset[0]:offset[0]+length[0], 
         offset[1]:offset[1]+length[1], 
         offset[2]:offset[2]+length[2]] = images[0][copyShape[0][0]:copyShape[0][1]+1,
                                                    copyShape[1][0]:copyShape[1][1]+1,
                                                    copyShape[2][0]:copyShape[2][1]+1,]
images[0] = newImage

labels[0] = sitk.GetArrayFromImage(labels[0])
newLabels = np.zeros((shape[0][1] - shape[0][0] + 1, shape[1][1] - shape[1][0] + 1, shape[2][1] - shape[2][0] + 1,), dtype="uint8", order="F")
newLabels[offset[0]:offset[0]+length[0], 
          offset[1]:offset[1]+length[1], 
          offset[2]:offset[2]+length[2]] = labels[0][copyShape[0][0]:copyShape[0][1]+1,
                                                     copyShape[1][0]:copyShape[1][1]+1,
                                                     copyShape[2][0]:copyShape[2][1]+1,]
labels[0] = newLabels

Reading 9005132.nii.gz. 1/60
Reading 9092628.nii.gz. 2/60
Reading 9131266.nii.gz. 3/60


In [5]:
from time import time
from sklearn.linear_model import Lasso

def applySRC(segImage, images, labels, numOfLabels, P, N, lassoTol=0.01, lassoMaxIter=1e4, 
                      verboseX=True, verboseY=True, xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1):  
# Perform SRC
    size = segImage.shape
    assert (size == images[0].shape), "Segmentation images and images has different size."
    assert (size == labels[0].shape), "Segmentation images and labels has different size."
    assert (len(images) == len(labels)), "Images and labels number mismatch."
    assert (numOfLabels >= 1), "Number of labels lower than one."
    
    # Allocate memory for the segmentation
    newSegmentation = np.zeros(shape=size, dtype='uint8')
    
    # x range
    if not (xmin >= N[0]//2 + P[0]//2 and xmin < size[0] - N[0]//2 - P[0]//2 - 1):
        xmin = N[0]//2 + P[0]//2
    if not (xmax >= N[0]//2 + P[0]//2 and xmax < size[0] - N[0]//2 - P[0]//2 - 1 and xmax > xmin):
        xmax = size[0] - N[0]//2 - P[0]//2 - 1
        
    # y range
    if not (ymin >= N[1]//2 + P[1]//2 and ymin < size[1] - N[1]//2 - P[1]//2 - 1):
        ymin = N[1]//2 + P[1]//2
    if not (ymax >= N[1]//2 + P[1]//2 and ymax < size[1] - N[1]//2 - P[1]//2 - 1 and ymax > ymin):
        ymax = size[1] - N[1]//2 - P[1]//2 - 1
        
    # z range
    if not (zmin >= N[2]//2 + P[2]//2 and zmin < size[2] - N[2]//2 - P[2]//2 - 1):
        zmin = N[2]//2 + P[2]//2
    if not (zmax >= N[2]//2 + P[2]//2 and zmax < size[2] - N[2]//2 - P[2]//2 - 1 and zmax > zmin):
        zmax = size[2] - N[2]//2 - P[2]//2 - 1
    
    
    for x in range(xmin, xmax):
        if verboseX:
            tstartX = time()
        for y in range(ymin, ymax):
            if verboseY:
                print('\tX:', x, '\tY:', y , end=" ")
                tstartY = time()

            for z in range(zmin, zmax):
                v = [x, y, z]

                L = createL(labels, v, N)

                if np.all(L == L[0]):
                    newSegmentation[x, y, z] = L[0]

                    continue

                A = createA(images, v, P, N)
                B = createB(segImage, v, P)

                if np.any(B != 0):
                    lasso = Lasso(alpha=np.amax(A.T @ np.array(B, dtype=np.float32)) * lassoTol, positive=True, max_iter=lassoMaxIter)

                    lasso.fit(A,B)
                    alpha = lasso.coef_
                    
                    residuals = []
                    for ll in range(numOfLabels + 1):
                        alphaJ = np.copy(alpha)
                        alphaJ[L != ll] = 0
                        residuals.append(np.linalg.norm(B - A @ alphaJ))

                    newSegmentation[x, y, z] = min(enumerate(residuals), key=lambda x: x[1])[0]
                else:
                    newSegmentation[x, y, z] = 0

                

            if verboseY:
                print("\tTime:", time() - tstartY)
        if verboseX:
                print("X:", x, "\tTime:", time() - tstartX)
    return newSegmentation

In [6]:
# Make to the proper type
images = np.array(images, order='C')
labels = np.array(labels, order='C')
P = np.array(P, dtype=np.int32)
N = np.array(N, dtype=np.int32)

In [7]:
segmentation = applySRC(images[0], images[1:], labels[1:], 5, P, N, lassoTol=0.00001, lassoMaxIter=1e4, verboseX=True, 
                        verboseY=False, xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=60, zmax=61)

/home/seiras/.local/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.024902728777751904, tolerance: 0.012696296296296298
  positive)


X: 3 	Time: 3.809413433074951
X: 4 	Time: 5.106277942657471
X: 5 	Time: 1.2645323276519775
X: 6 	Time: 0.39407873153686523
X: 7 	Time: 0.44066643714904785
X: 8 	Time: 0.40122175216674805
X: 9 	Time: 0.4163246154785156
X: 10 	Time: 0.3697659969329834
X: 11 	Time: 0.3982887268066406
X: 12 	Time: 0.36536145210266113
X: 13 	Time: 0.3360164165496826
X: 14 	Time: 0.32723331451416016
X: 15 	Time: 0.3463890552520752
X: 16 	Time: 0.3377394676208496
X: 17 	Time: 0.37166714668273926
X: 18 	Time: 0.3427090644836426
X: 19 	Time: 0.33007192611694336
X: 20 	Time: 0.36566710472106934
X: 21 	Time: 0.3291957378387451


/home/seiras/.local/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


X: 22 	Time: 0.45005011558532715
X: 23 	Time: 0.3322300910949707
X: 24 	Time: 0.3306436538696289
X: 25 	Time: 0.3113701343536377
X: 26 	Time: 0.31521034240722656
X: 27 	Time: 0.3093411922454834
X: 28 	Time: 0.30446457862854004


/home/seiras/.local/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


X: 29 	Time: 0.4273953437805176
X: 30 	Time: 0.3293483257293701
X: 31 	Time: 0.34311771392822266
X: 32 	Time: 0.31467342376708984
X: 33 	Time: 0.33469676971435547
X: 34 	Time: 0.3325660228729248
X: 35 	Time: 0.3255190849304199
X: 36 	Time: 0.2873678207397461
X: 37 	Time: 0.2709834575653076
X: 38 	Time: 0.35700392723083496
X: 39 	Time: 0.35136985778808594
X: 40 	Time: 0.22891020774841309
X: 41 	Time: 0.2752256393432617
X: 42 	Time: 0.25763702392578125
X: 43 	Time: 0.2771182060241699
X: 44 	Time: 0.2587604522705078
X: 45 	Time: 0.2505345344543457
X: 46 	Time: 0.22090792655944824
X: 47 	Time: 0.2317352294921875
X: 48 	Time: 0.21737170219421387
X: 49 	Time: 0.2749159336090088
X: 50 	Time: 0.23609185218811035
X: 51 	Time: 0.2810499668121338
X: 52 	Time: 0.28135037422180176
X: 53 	Time: 0.27684831619262695
X: 54 	Time: 0.29204249382019043
X: 55 	Time: 0.2926595211029053
X: 56 	Time: 0.2793591022491455
X: 57 	Time: 0.24841618537902832
X: 58 	Time: 0.24720358848571777
X: 59 	Time: 0.2542028427

X: 279 	Time: 0.26111578941345215
X: 280 	Time: 0.24603605270385742
X: 281 	Time: 0.2504303455352783
X: 282 	Time: 0.23947548866271973
X: 283 	Time: 0.24518680572509766
X: 284 	Time: 0.2377023696899414
X: 285 	Time: 0.26230621337890625
X: 286 	Time: 0.23276257514953613
X: 287 	Time: 0.26162004470825195
X: 288 	Time: 0.25400376319885254
X: 289 	Time: 0.2490401268005371
X: 290 	Time: 0.25482845306396484
X: 291 	Time: 0.2689659595489502
X: 292 	Time: 0.26150941848754883
X: 293 	Time: 0.2563192844390869
X: 294 	Time: 0.2663707733154297
X: 295 	Time: 0.26450371742248535
X: 296 	Time: 0.28043293952941895
X: 297 	Time: 0.2666311264038086
X: 298 	Time: 0.2544724941253662
X: 299 	Time: 0.2553982734680176
X: 300 	Time: 0.2734508514404297
X: 301 	Time: 0.25682687759399414
X: 302 	Time: 0.2679173946380615
X: 303 	Time: 0.2786705493927002
X: 304 	Time: 0.27454185485839844
X: 305 	Time: 0.2773110866546631
X: 306 	Time: 0.2899301052093506
X: 307 	Time: 0.3060948848724365
X: 308 	Time: 0.2838671207427

In [8]:
from cGen import cGen

segmentation2 = cGen.applySRC(images[0], images[1:], labels[1:], 5, P, N, lassoTol=0.00001, lassoMaxIter=1e4, verboseX=True, 
                        verboseY=False, xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=60, zmax=61)

/home/seiras/.local/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.024902728777751904, tolerance: 0.012696296296296298
  positive)


X: 3 	Time: 3.036905288696289
X: 4 	Time: 4.356823682785034
X: 5 	Time: 0.9885249137878418
X: 6 	Time: 0.13395047187805176
X: 7 	Time: 0.18381571769714355
X: 8 	Time: 0.12873530387878418
X: 9 	Time: 0.1620042324066162
X: 10 	Time: 0.09675812721252441
X: 11 	Time: 0.10546231269836426
X: 12 	Time: 0.1101388931274414
X: 13 	Time: 0.09767675399780273
X: 14 	Time: 0.0687255859375
X: 15 	Time: 0.08809661865234375
X: 16 	Time: 0.08547663688659668
X: 17 	Time: 0.07448792457580566
X: 18 	Time: 0.05891561508178711
X: 19 	Time: 0.05841469764709473
X: 20 	Time: 0.09310126304626465
X: 21 	Time: 0.0743563175201416


/home/seiras/.local/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


X: 22 	Time: 0.1966860294342041
X: 23 	Time: 0.0722038745880127
X: 24 	Time: 0.08214354515075684
X: 25 	Time: 0.07660508155822754
X: 26 	Time: 0.07324528694152832
X: 27 	Time: 0.06418943405151367
X: 28 	Time: 0.06883454322814941
X: 29 	Time: 0.17807340621948242


/home/seiras/.local/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


X: 30 	Time: 0.0933685302734375
X: 31 	Time: 0.0815436840057373
X: 32 	Time: 0.06515383720397949
X: 33 	Time: 0.0896153450012207
X: 34 	Time: 0.09363508224487305
X: 35 	Time: 0.10652041435241699
X: 36 	Time: 0.08947062492370605
X: 37 	Time: 0.05349850654602051
X: 38 	Time: 0.03842520713806152
X: 39 	Time: 0.044092416763305664
X: 40 	Time: 0.04099535942077637
X: 41 	Time: 0.06208205223083496
X: 42 	Time: 0.06463122367858887
X: 43 	Time: 0.08641242980957031
X: 44 	Time: 0.08194470405578613
X: 45 	Time: 0.08101773262023926
X: 46 	Time: 0.06294870376586914
X: 47 	Time: 0.07588005065917969
X: 48 	Time: 0.06592607498168945
X: 49 	Time: 0.08581376075744629
X: 50 	Time: 0.08633971214294434
X: 51 	Time: 0.1191718578338623
X: 52 	Time: 0.11166238784790039
X: 53 	Time: 0.10898518562316895
X: 54 	Time: 0.13736748695373535
X: 55 	Time: 0.1453533172607422
X: 56 	Time: 0.1215667724609375
X: 57 	Time: 0.10929584503173828
X: 58 	Time: 0.10917425155639648
X: 59 	Time: 0.12648439407348633
X: 60 	Time: 0.

X: 279 	Time: 0.06941843032836914
X: 280 	Time: 0.05794715881347656
X: 281 	Time: 0.05952811241149902
X: 282 	Time: 0.04584455490112305
X: 283 	Time: 0.055817365646362305
X: 284 	Time: 0.060628652572631836
X: 285 	Time: 0.05378150939941406
X: 286 	Time: 0.04176449775695801
X: 287 	Time: 0.06125974655151367
X: 288 	Time: 0.051698923110961914
X: 289 	Time: 0.04125070571899414
X: 290 	Time: 0.044342041015625
X: 291 	Time: 0.046904802322387695
X: 292 	Time: 0.04680204391479492
X: 293 	Time: 0.03850364685058594
X: 294 	Time: 0.04262804985046387
X: 295 	Time: 0.05359196662902832
X: 296 	Time: 0.04679465293884277
X: 297 	Time: 0.059076547622680664
X: 298 	Time: 0.04366278648376465
X: 299 	Time: 0.036162376403808594
X: 300 	Time: 0.035770416259765625
X: 301 	Time: 0.033010244369506836
X: 302 	Time: 0.037270545959472656
X: 303 	Time: 0.05251646041870117
X: 304 	Time: 0.04302668571472168
X: 305 	Time: 0.04084610939025879
X: 306 	Time: 0.06469035148620605
X: 307 	Time: 0.05530285835266113
X: 308 

In [9]:
from utils import Dice
dice = Dice(labels[0], segmentation, 5)
for i, d in enumerate(dice):
    print(f'Label {i+1}: {d}')

Label 1: 0.017535899097451586
Label 2: 0.009502864092659227
Label 3: 0.023738675344113735
Label 4: 0.0
Label 5: 0.0


In [10]:
from utils import Dice
dice = Dice(labels[0], segmentation2, 5)
for i, d in enumerate(dice):
    print(f'Label {i+1}: {d}')

Label 1: 0.017526062463577304
Label 2: 0.009515227514761205
Label 3: 0.023733486130350274
Label 4: 0.0
Label 5: 0.0


In [11]:
from PIL import Image as image

def showImg_(img, z=60):
    from PIL import Image as image
    a = sitk.GetArrayFromImage(img[z,:,:])
    disImg = image.fromarray(np.interp(a, (a.min(), a.max()), (0, 255)).astype('uint8'))
    disImg.show()
    
def showImg2_(img, z=60):
    disImg = image.fromarray(sitk.GetArrayFromImage(img[z,:,:]).astype('uint8'))
    disImg.show()
    
def showImg3_(img, z=60):
    from PIL import Image as image
    a = sitk.GetArrayFromImage(img[z,:,:])
    disImg = image.fromarray(np.interp(a, (0, 5000), (0, 255)).astype('uint8'))
    disImg.show()
    
def showImg4_(img, z=60):
    img2 = image.fromarray(sitk.GetArrayFromImage(img[z,:,:])*60, 'L')
    img2.show()

In [14]:
showImg4_(sitk.GetImageFromArray(segmentation), 60)
showImg4_(sitk.GetImageFromArray(segmentation2), 60)

In [13]:
showImg4_(sitk.GetImageFromArray(segmentation2), 60)
showImg4_(sitk.GetImageFromArray(labels[1]), 60)